<a href="https://colab.research.google.com/github/nisanuro/CNG483-Project1/blob/master/CNG483_Project1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
from google.colab import output
from google.colab import drive
import concurrent.futures
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_images(path):
    images = []

    for filename in os.listdir(path):
        img=cv2.imread(os.path.join(image_path, filename),cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)

    return images

In [0]:
"""
def histogram(image, b):
    bins = []
    width = int(256/b)

    for low in range(0, b * width, width):
        bins.append((low, low+width))

    hist = np.zeros(b).astype('int32')
    image = np.array(image).flatten()

    for value in image:
        for i in range(0, len(bins)):
            if (bins[i][0] <= value < bins[i][1]):
                hist[i] += 1
    
    return hist
"""

def histogram(image,b):
    bins = np.array([i for i in range(0, 256, int(256/b))])
    image = np.array(image).flatten()
    inds = np.digitize(image,bins, right=False)
    temp = np.array([(inds[i]-1) for i in range(0, len(inds))])
    count = np.bincount(temp)
    
    hist = np.array([count[i] for i in range(0, len(count), int(256/b))])
    if len(hist) != b:
        print("b: ", b, "len hist: ", len(hist))
        z = np.zeros(b-len(hist)).astype('int64')
        hist = np.concatenate((hist, z))
    print("hist: ",  len(hist))
    #count = count[::int(256/b)]
    #print(count)
    #print(bins)
    return hist
  

In [0]:
def color_histogram(img, b):
    chans = cv2.split(img)
    colors = ("b", "g", "r")

    features = []
   
    for (chan, color) in zip(chans, colors):
        if color == 'b':
            bf = histogram(chan, b)
        elif color == 'g':
            gf = histogram(chan, b)
        else:
            rf = histogram(chan, b)
    
    # combination of blue and red channel bins
    temp = np.transpose([np.tile(bf, len(gf)), np.repeat(gf, len(bf))])
    
    blue_green_hist = []    
    for i in temp:
        blue_green_hist.append(sum(i))

    # same operation with red channel bins
    temp = np.transpose([np.tile(rf, len(blue_green_hist)), np.repeat(blue_green_hist, len(rf))])
    """
    temp = np.array(np.meshgrid(bf, gf, rf)).T.reshape(-1,3)
    """
    print("temp:")
    print(temp.shape)

    color_hist = []    
    for i in temp:
        color_hist.append(sum(i))
    color_hist = np.array(color_hist)
    print("color hist: ")
    print(color_hist.shape)
    return color_hist

In [0]:
def split_image_into_grids(image, grid):

    grids = []

    height=image.shape[0]
    width=image.shape[1]

    M = height//grid
    N = width//grid

    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M
            x1 = x + N
            grids.append(image[y:y+M, x:x+N])


    #cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0))

    #os.chdir("/content/drive/My Drive/SaveDeneme")
    #cv2.imwrite("im" + str(i) + '_' + str(x) + '_' + str(y)+".png",tiles)
    '''
    plt.figure()
    plt.imshow(grid) 
    plt.show()''' 

    return np.array(grids)

     

In [0]:
def create_dataset(path):
    print("create dataset")
    class_paths = []
    images = []
    labels = []

    with os.scandir(path) as itr: 
        for subdir in itr : 
            if subdir.is_dir():
                p = path + '/' + subdir.name
                class_paths.append(p)
    
    for p in class_paths:
        #print(p)
        for filename in os.listdir(p):
            if filename.endswith(".jpg"):
                labels.append(os.path.basename(os.path.normpath(p)))                
                img = cv2.imread(os.path.join(p, filename))

                if img is not None:
                    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
   
    return images, labels


In [0]:
def threaded_feature_extraction(imageslabels, grid, bin, isRGB):
    print("feature extraction")
    features = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futuretoimage = {executor.submit(feature_extract_single, imagelabel, grid, bin, isRGB): imagelabel for imagelabel in imageslabels}
        counter = 0
        for future in concurrent.futures.as_completed(futuretoimage):
            features.append(future.result())
            counter += 1
            print(counter)
    return features

def feature_extract_single(imagelabel, grid, b, isRGB):
    image = imagelabel[0]
    label = imagelabel[1]
    #print(label)
    i = cv2.resize(image, (256, 256))
    i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
    grid_hists = []
    grids = split_image_into_grids(i, grid)
    #print("grids: ")
    #print(len(grids))
    for i in grids:
        if(isRGB):
            grid_hists.append(color_histogram(i, b))
        else:
            i = cv2.cvtColor(i, cv2.COLOR_RGB2GRAY)
            grid_hists.append(histogram(i, b))
    '''
    # Concatenation
    grids_flat = np.array(grid_hists).flatten()
    '''

    # Sum
    if grid != 1:
        grids_flat = [ sum(x) for x in zip(*grid_hists) ]
        grids_flat = np.array(grids_flat)
        print("grids: ", len(grids_flat))
        return (grids_flat, label)
    else:
        print("grids: ", len(grid_hists))
        return (np.array(grid_hists).flatten(), label)

    


In [133]:
"""def feature_extraction(images, grid, bin, isRGB):
    print("feature extraction")
    features = []
    size = len(images)
    for i, j in zip(images, range(size)):
        print(j)
        i = cv2.resize(i, (256, 256))
        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        grids = split_image_into_grids(i, grid)

        grid_hists = []

        for i in grids:
            if(isRGB):
                grid_hists.append(color_histogram(i, bin))
            else:
                grid_hists.append(histogram(i, bin))
        
        # Concatenation
        grids_flat = np.array(grid_hists).flatten()
        
        '''
        # Sum
        grids_flat = np.array(grid_hists)
        grids_flat = np.sum(grids_flat, axis=0)
        '''
            
        features.append(grids_flat)
        
    return features"""


'def feature_extraction(images, grid, bin, isRGB):\n    print("feature extraction")\n    features = []\n    size = len(images)\n    for i, j in zip(images, range(size)):\n        print(j)\n        i = cv2.resize(i, (256, 256))\n        i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)\n        grids = split_image_into_grids(i, grid)\n\n        grid_hists = []\n\n        for i in grids:\n            if(isRGB):\n                grid_hists.append(color_histogram(i, bin))\n            else:\n                grid_hists.append(histogram(i, bin))\n        \n        # Concatenation\n        grids_flat = np.array(grid_hists).flatten()\n        \n        \'\'\'\n        # Sum\n        grids_flat = np.array(grid_hists)\n        grids_flat = np.sum(grids_flat, axis=0)\n        \'\'\'\n            \n        features.append(grids_flat)\n        \n    return features'

In [0]:
def classification(trainI, trainL, testI, testL, k):
    print("classification")
    model = KNeighborsClassifier(n_neighbors=k, metric='euclidean', algorithm='brute', n_jobs=-1)
    model.fit(trainI, trainL)
    acc = model.score(testI, testL)
    print(acc)

In [135]:
if __name__ == "__main__":

    test_path = "/content/drive/My Drive/CNG483-Project 1/TestSet"
    train_path = "/content/drive/My Drive/CNG483-Project 1/TrainingSet"
    validation_path = "/content/drive/My Drive/CNG483-Project 1/ValidationSet"

    
    # testI, testL = create_dataset(test_path)
    trainI, trainL = create_dataset(train_path)
    validationI, validationL = create_dataset(validation_path)
    
    # grid --> 1, 2, 4
    # bins --> 1, 128, 256
    # k --> 1, 5, 10
    # isRGB --> True, False

    grid = 2
    bins = 256
    isRGB = False
    k = 5

create dataset
create dataset


In [136]:
    imagelabels = []
    for i,l in zip(trainI, trainL):
        imagelabels.append((i,l))
    trainI_f = threaded_feature_extraction(imagelabels, grid, bins, isRGB)
    #trainI_f = threaded_feature_extraction(trainI, grid, bins, isRGB)
    trainf = []
    trainl = []
    for (i,l) in trainI_f:
        trainf.append(i)
        trainl.append(l)

    imagelabels = []
    for i,l in zip(validationI, validationL):
        imagelabels.append((i,l))
    
    validationI_f = threaded_feature_extraction(imagelabels, grid, bins, isRGB)
    validationf = []
    validationl = []
    for (i,l) in validationI_f:      
        validationf.append(i)
        validationl.append(l)

    #validationI_f = feature_extraction(validationI, grid, bins, isRGB)
    
    trainf = np.array(trainf)
    trainl = np.array(trainl)
    validationf = np.array(validationf)
    validationl = np.array(validationl)


Streaming output truncated to the last 5000 lines.
hist:  256
grids:  256
272
273
274
275
276
256
hist:  256
hist:  256
grids:  256277
b:  256 hist:  256

len hist:  227
278
b:  256 len hist:  220
b:  256 b:  256 len hist:  232
hist:  256
b:  b:  256 len hist:  236
hist:  b:  256 len hist:  217
hist: 256
256hist:  len hist: hist: hist:  256
256
b:  256 len hist:  256
grids:  256grids:  b:   194
hist:  256
grids:  256
 212
hist:  256b:  256 len hist:  203
hist:  256
256
b:  256 len hist:  188
hist:  256

 b:  256 len hist:  
256
grids:  256
b:  256 len hist:  166
hist:  256
b:  256256 len hist:  236243b:  256 len hist:  215

hist:  256
hist:  256
b:  256 len hist:  b:  256 len hist:  216
hist:  256
 224 
hist:  256
len hist:  223

hist: len hist:  217
hist:  256279
280
281
282
b:  256 len hist:  201
hist:  256
hist: grids:  256
 256
283
b:  256 len hist:  183
hist: b:  hist:  256
b:  256 len hist:   256

b:  256 len hist:  240
hist:  256
grids:  256
b:  256 len hist:  256
284
b:  256b: 

In [137]:
    validationf


array([[    1,     9,    20, ...,   298,   722,   480],
       [    0,     1,     2, ...,  1154,  2992, 22907],
       [    0,     2,     3, ...,  3198,  4856, 31976],
       ...,
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [ 2773,  1370,   735, ...,     0,     0,     0]])

In [138]:
    print(trainf.dtype)
    print(validationf.dtype)
    print(trainl.dtype)

int64
int64
<U10


In [139]:
    classification(trainf, trainl, validationf, validationl, k)
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

classification
0.3904761904761905
